In [1]:
import pickle
from rank_bm25 import BM25Okapi
from konlpy.tag import Mecab
tokenizer = Mecab()

In [2]:
from konlpy.tag import Mecab

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [3]:
tokenizer = Mecab()

In [4]:
import json
from glob import glob
def load_json_data(data_dir):
    """
    Load multiple JSON files from the folder and merge.
    """

    files = glob(data_dir+"/*.json")
    files.sort()
    all_data = []
    for file_path in files:
        #print("Loading: ",file)
        #file_path = os.path.join(data_dir, file)
        with open(file_path, "r", encoding = "utf-8-sig") as f:
            doc = json.load(f)
        all_data.append(doc)
        #all_data += doc
    return all_data


In [62]:
all_docs = load_json_data('/Project/json_data/')

documents = []
for item in all_docs:
    try:
        documents.append(item['title'] + item['contents'])
    except:
        continue
        documents.append(item['title'] + item['content'])
        

In [63]:
tokenized_documents = [tokenizer.morphs(doc) for doc in documents]

In [64]:
# Create a BM25 index
bm25 = BM25Okapi(tokenized_documents)

# Save the BM25 index to a file
with open('/Project/index_save_dir/sparse/bm25_index.pickle', 'wb') as index_file:
    pickle.dump(bm25, index_file)

In [65]:
# Load the BM25 index from a file
with open('/Project/index_save_dir/sparse/bm25_index.pickle', 'rb') as index_file:
    bm25 = pickle.load(index_file)

In [66]:
# Query
query = "유럽 리그에서 활동중인 한국인 선수 이름은?"

# Tokenize the query
tokenized_query = tokenizer.morphs(query)

# Get the most relevant document indices
doc_scores = bm25.get_scores(tokenized_query)

In [67]:
import numpy as np
most_relevant_index = doc_scores.argmax()
top_k = 5
most_relevant_index = np.argpartition(doc_scores,-1*top_k)[-1*top_k:]

# Retrieve the most relevant passage

for i in most_relevant_index:
  print("Most relevant passage:",documents[i])

Most relevant passage: 이강인, 슛돌이에서 최연소 외국인 선수로…최초 U20 월드컵 MVP도
/이강인 인스타그램 캡처[앵커]이강인 선수는  올해 스페인  프리메라리가에서 두 자릿수  공격포인트를 기록하며 리그 정상급 선수로  발돋움했습니다.  지난 2019년 열린 U20 월드컵에선  대한민국 남자선수로는  최초로 골든볼을  수상하기도 했습니다.  김창섭 기자입니다. [리포트]이강인은 지난 2007년 한 축구 예능 프로그램을 통해 대중에 이름을 알렸습니다. 이후 스페인으로 넘어가  발렌시아 유소년 아카데미에 입단해 유럽에서 축구 선수 생활을 시작했습니다. 이강인"커서 형들과 같이 우리나라를 잘 이끌어서 좋은 선수가 되겠습니다." 2018년 발렌시아 1군으로 성인 무대에 데뷔했는데, 당시 나이는 만 17세 8개월 11일로 발렌시아 구단 역사상  최연소 외국인 선수로  기록된 바 있습니다. 이후 이강인은  2021년 마요르카로 이적한 뒤 벤치 선수에서 주전 선수로  도약을 거듭하게 됩니다. 이번 시즌  스페인 프리메라리가에서만 36경기를 출전해 6골 6도움으로  한국인 최초로 라리가에서 두 자릿수 공격 포인트를 기록하기도 했습니다.  약점으로 지적됐던  수비에서도 발군의 활약을 보이며 라리가 올해의 미드필더 후보에  오르기도 했습니다. 국가대표에서  이강인의 존재감은 더욱 큽니다. 지난 2019년 U20 월드컵에선 형들과 함께 출전해  팀의 준우승을 이끌었고, FIFA 주관 대회 최초의  한국인 골든볼 수상자로 이름을 남겼습니다.? 이강인"앞으로도 더 좋은 선수가 돼서 대한민국 대표 선수로서 더 좋은 결과 나오고 좋은 모습 보여드리도록 노력하겠습니다" 지난해 열린 카타르 월드컵에서도 막판 대표팀에 승선해  조규성의 그림 같은 헤딩골을  이끌어 내기도 했습니다. TV조선 김창섭입니다.
						
						
						
						
						
Most relevant passage: ‘학업+축구 병행 도전’ 강민우, 캐나다 대학-세미프로 입단
이

In [34]:
! pip install farm-haystack

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: torch-tensorrt 1.0.0a0 has a non-standard dependency specifier torch>=1.9.0<1.11.0. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torch-tensorrt or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from transformers import AutoModel, AutoTokenizer, BertModel
import torch
from copy import deepcopy

class KobertBiEncoder(torch.nn.Module):
    def __init__(self):
        super(KobertBiEncoder, self).__init__()
        
        self.passage_encoder = BertModel.from_pretrained("skt/kobert-base-v1")     
        self.query_encoder = BertModel.from_pretrained("skt/kobert-base-v1")
            
    def forward(
        self, input_ids: torch.LongTensor, attention_mask: torch.LongTensor, type: str
    ) -> torch.FloatTensor:
        """passage 또는 query를 bert로 encoding합니다."""

        if type == "passage":
            return self.passage_encoder(
                input_ids=input_ids, attention_mask=attention_mask
            ).pooler_output
        else:
            return self.query_encoder(
                input_ids=input_ids, attention_mask=attention_mask
            ).pooler_output

    def checkpoint(self, model_ckpt_path):
        torch.save(deepcopy(self.state_dict()), model_ckpt_path)

    def load(self, model_ckpt_path):
        with open(model_ckpt_path, "rb") as f:
            state_dict = torch.load(f)
        self.load_state_dict(state_dict, strict=False)

In [3]:
import torch
from kobert_tokenizer import KoBERTTokenizer

check_point_dir = '/Project/2050iter_model.pt'
model = KobertBiEncoder()


tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1")
model.load("/Project/2050iter_model.pt")

model.cuda()
model.eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


KobertBiEncoder(
  (passage_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [4]:
from glob import glob
import json
import os
from torch.utils.data import DataLoader

batch_size = 32 

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        text = self.texts[index]
        tokenized_text = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length)
        input_ids = torch.tensor(tokenized_text["input_ids"])
        attention_mask = torch.tensor(tokenized_text["attention_mask"])
        return input_ids, attention_mask
    
def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    attention_masks = [item[1] for item in batch]
    input_ids = torch.stack(input_ids, dim=0)
    attention_masks = torch.stack(attention_masks, dim=0)
    return input_ids, attention_masks

def load_json_data(data_dir):
    """
    Load multiple JSON files from the folder and merge.
    """

    files = glob(data_dir+"/*.json")
    files.sort()
    all_data = []
    for file_path in files:
        #print("Loading: ",file)
        #file_path = os.path.join(data_dir, file)
        with open(file_path, "r", encoding = "utf-8-sig") as f:
            doc = json.load(f)
        all_data.append(doc)
        #all_data += doc
    return all_data

data_dir = '/Project/json_data'
all_docs = load_json_data(data_dir)
use_content_type = 'All'

if use_content_type == "title":
    doc_content = [item['title'] for item in  all_docs]
elif  use_content_type == "content":
    doc_content = [item['contents'] for item in  all_docs]
else:
    doc_content = [item['title'] + item['contents'] for item in  all_docs]
    
doc_dataset = TextDataset(doc_content, tokenizer)
doc_loader = DataLoader(doc_dataset, 
                        batch_size=batch_size, 
                        collate_fn=collate_fn,
                        shuffle = False
                        )



In [30]:
from tqdm import tqdm

doc_embedding = []
for batch in tqdm(doc_loader):
    batch = tuple(t.cuda() for t in batch)
    with torch.no_grad():
        output = model(input_ids = batch[0], attention_mask = batch[1], type = 'passage')
    doc_embedding.append(output)
doc_embedding = torch.cat(doc_embedding, dim=0)
print("Finish converting embeddings.")

100%|██████████| 850/850 [07:20<00:00,  1.93it/s]


Finish converting embeddings.
Start building faiss index...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 hidden_dim = doc_embedding.shape[1]                                                         │
│   15 dense_index = faiss.IndexFlatL2(hidden_dim)                                                 │
│   16 dense_index.add(doc_embedding.cpu().numpy())                                                │
│ ❱ 17 faiss.write_index(dense_index,faiss_save_path)                                              │
│   18 print("Finish building index.")                                                             │
│   19                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'faiss_save_path' is not defined

In [5]:
import faiss

faiss_save_path = '/Project/index_save_dir/dense/faiss.index'

# Build faiss index by using doc embedding
print("Start building faiss index...")
hidden_dim = doc_embedding.shape[1]
dense_index = faiss.IndexFlatL2(hidden_dim)
dense_index.add(doc_embedding.cpu().numpy())
faiss.write_index(dense_index,faiss_save_path)
print("Finish building index.")

Start building faiss index...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4                                                                                             │
│    5 # Build faiss index by using doc embedding                                                  │
│    6 print("Start building faiss index...")                                                      │
│ ❱  7 hidden_dim = doc_embedding.shape[1]                                                         │
│    8 dense_index = faiss.IndexFlatL2(hidden_dim)                                                 │
│    9 dense_index.add(doc_embedding.cpu().numpy())                                                │
│   10 faiss.write_index(dense_index,faiss_save_path)                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'doc_embedding' is not defined

In [6]:
index_save_dir = faiss_save_path
dense_index = faiss.read_index(index_save_dir)

In [7]:
query = '호날두의 소속팀은?'

top_k = 5
tokenized_text  = tokenizer(query, padding = 'max_length', truncation = True,
                                max_length = 512, return_tensors = "pt")

input_ids = tokenized_text["input_ids"].cuda()
attention_mask = tokenized_text["attention_mask"].cuda()

with torch.no_grad():
    query_embed = model(input_ids = input_ids, attention_mask = attention_mask, type = 'query')

query_embed = query_embed.detach().cpu().numpy()

_,doc_id = dense_index.search(query_embed, top_k)

cnt = 0
raw_reference_list = []

for i,id in enumerate(doc_id[0]):
    raw_reference = all_docs[id]        
    raw_reference_list.append(raw_reference)

    cnt += 1
    if (cnt == top_k) :
        break

In [8]:
raw_reference_list

[{'title': "PSG 이강인, '발렌시아 인연' 솔레르와 훈련 호흡 눈길",
  'url': 'https://sports.news.naver.com/news?oid=003&aid=0011970693',
  'contents': "\n패스훈련·미니게임 등으로 적응 이어가[서울=뉴시스]PSG 구단은 13일(한국시간) 홈페이지를 통해 이강인이 합류한 후, 이틀째 훈련 모습을 약 3분짜리 영상으로 담아 공개했다. 이강인(왼쪽)과 솔레르가 스트레칭을 하고 있다. (사진 = PSG 홈페이지 캡처) *재판매 및 DB 금지[서울=뉴시스] 박지혁 기자 = 프랑스 프로축구 명문 파리 생제르맹(PSG) 유니폼을 입은 이강인(22)이 적응에 한창이다.PSG 구단은 13일(한국시간) 홈페이지를 통해 이강인이 합류한 후 이틀째 훈련 모습을 약 3분짜리 영상으로 담아 공개했다.전날 세계적인 스타 공격수 네이마르(브라질)와 나란히 마사지 침대에 자리한 모습이 담겨 화제를 모은 데 이어 이날은 '발렌시아 인연' 카를로스 솔레르(26·스페인)와 호흡이 눈길을 끌었다.스페인 국가대표 미드필더인 솔레르는 이강인보다 네 살 위로 지난해 2022 카타르월드컵에서 골을 터뜨렸던 실력파다. 이강인과는 과거 발렌시아(스페인)에서 한솥밥을 먹었다.솔레르는 발렌시아 유스 출신으로 2015년부터 지난해까지 발렌시아에서 뛰다가 PSG로 이적했다.이강인이 2011년 발렌시아 유스에 합류할 때부터 함께 해 2021년 마요르카(스페인)로 이적할 때까지 사실상 청소년기를 함께 보낸 진한 인연이다.둘은 밝은 표정으로 함께 훈련장에 입장하고, 스트레칭을 나란히 하며 계속 대화를 주고받았다. 훈련 이후 그라운드를 빠져나갈 때도 함께 했다. 이강인은 스페인어에 능통하다.이날 훈련에서 이강인은 강도 높은 패스 훈련과 미니 게임으로 구슬땀을 흘렸다. 루이스 엔리케 감독은 선수들을 독려하며 적극적으로 훈련을 이끌었다.전날 이강인과 나란히 있던 네이마르는 이날 영상에 보이지 않았다. 발목 부상 후유증으로 재활과 회복에 집중하